In [120]:
import pandas as pd
import json
import time
import math

In [151]:
# https://stackoverflow.com/questions/5849800
def tic():
    #Homemade version of matlab tic and toc functions
    global startTime_for_tictoc
    startTime_for_tictoc = time.time()


def toc():
    if 'startTime_for_tictoc' in globals():
        print("Elapsed time is " + str(time.time() - startTime_for_tictoc) + " seconds.")
    else:
        print("Toc: start time not set")


def read_json_data(filename):
    df_adaptive = pd.read_json(filename)
    return df_adaptive


def define_predicates(data):

    P = set()

    # For bibliography, use one of the following fields:
    # author, title, year, venue, other
    # Although it sounds odd, the paper uses every field for every predicate
    applicable_fields = ['authors','title','year','venue','other']
    
    # Contains common integer
    name = 'Contains common integer'
    h = lambda x    : "".join([c if c.isdigit() else ' ' for c in x]).split()
    p_= lambda x, y : bool(set(x) & set(y))
    for p in apply_predicate_to_data(name, h, p_, applicable_fields, data):
        P.add(p)
    
    # Contains same or off-by-one integer
    name = 'Contains same or off-by-one integer'
    h = lambda x    : map(int, "".join([c if c.isdigit() else ' ' for c in x]).split())
    p_= lambda x, y : bool(set(x) & set(y)) or contains_off_by_one(x,y)
    def contains_off_by_one(x, y):
        for int1 in x:
            for int2 in y:
                if abs(int1 - int2) == 1:
                    return True
        return False
    for p in apply_predicate_to_data(name, h, p_, applicable_fields, data):
        P.add(p)
    
    # Exact match
    name = 'Exact match'
    h = lambda x    : str(x).strip()
    p_= lambda x, y : x == y
    for p in apply_predicate_to_data(name, h, p_, applicable_fields, data):
        P.add(p)

    # Contains common token
    name = 'Contains common token'
    h = lambda x    : str(x).strip().split()
    p_= lambda x, y : bool(set(x) & set(y))
    for p in apply_predicate_to_data(name, h, p_, applicable_fields, data):
        P.add(p)

    # Same 3 first characters
    name = 'Same 3 first chars'
    h = lambda x    : str(x).strip()
    p_= lambda x, y : x[:3] == y[:3]
    for p in apply_predicate_to_data(name, h, p_, applicable_fields, data):
        P.add(p)

    # Same 5 first characters
    name = 'Same 5 first chars'
    h = lambda x    : str(x).strip()
    p_= lambda x, y : x[:5] == y[:5]
    for p in apply_predicate_to_data(name, h, p_, applicable_fields, data):
        P.add(p)

    # Same 7 first characters
    name = 'Same 7 first chars'
    h = lambda x    : str(x).strip()
    p_= lambda x, y : x[:7] == y[:7]
    for p in apply_predicate_to_data(name, h, p_, applicable_fields, data):
        P.add(p)
    
    return P


'''
Here's a potential data structure for a predicate:

class Predicate:
    name: a short english definition
    field: the field to which to apply
    h() : an indexing function
    p() : an equality function
    pairs_covered : a list of the pairs for which p() returns 1
    num_pos_pairs_covered() : should return the count of pairs_covered that are 1/Pos/True  in pair_labels
    num_neg_pairs_covered() : should return the count of pairs_covered that are 0/Neg/False in pair_labels
    weight() : get set cover weight
'''
class Predicate:

    def __init__(self, name, h, p, field, data):
        print("processing for predicate ({}), field ({})".format(name,field))
        self.name = name
        self.h = h
        self.p = p
        self.field = field
        self.pairs_covered = self._get_pairs_covered(data)

    def _get_pairs_covered(self, data):
        newcol = '({})_{}'.format(self.name, self.field)
        data[newcol] = data[self.field].apply(self.h)
        data['tmpkey'] = 1
        cols = ['id', newcol, 'tmpkey']
        merged = pd.merge(data[cols], data[cols], on='tmpkey')\
                   .drop('tmpkey',axis=1)\
                   .rename(columns={newcol+'_x':'x',newcol+'_y':'y'})
        f = lambda row : self.p(row['x'],row['y'])
        return merged.apply(f, axis=1)

    def num_pos_pairs_covered(self, pair_labels, active_pairs):
        counts = (self.pairs_covered & pair_labels & active_pairs).value_counts()
        if True in counts:
            return counts[True]
        else:
            print("Predicate {} apparently covers 0 positive pairs".format(self))
            return 0
    
    def get_pos_pairs_set(self, pair_labels, active_pairs):
        return set([index for index, is_true in enumerate(self.pairs_covered & pair_labels & active_pairs) if is_true])
    
    def num_neg_pairs_covered(self, pair_labels, active_pairs):
        counts = (p.pairs_covered & ~pair_labels & active_pairs).value_counts()
        if True in counts:
            return counts[True]
        else:
            print("Predicate {} apparently covers 0 negative pairs".format(self))
            return 0
    
    def set_weight(self, pair_labels, active_pairs):
        self.weight = self.num_neg_pairs_covered(pair_labels, active_pairs)
    
    def covers(self, pair_index):
        return self.pairs_covered[pair_index]
    
    def __str__(self):
        return '{}, field: {}'.format(self.name, self.field)


def apply_predicate_to_data(name, h, p_, applicable_fields, data):
    for field in applicable_fields:
        tic()
        p = Predicate(name, h, p_, field, data)
        yield p
        toc()

In [177]:
# Returns the number of positive pairs covered by predicates
def B_(predicates):
    global pair_labels
    result = pair_labels.copy()
    result[:] = False
    for p in predicates:
        result = result | p.pairs_covered
    return (result & pair_labels)

def R_(predicates):
    global pair_labels
    result = pair_labels.copy()
    result = False
    for p in predicates:
        result = result | p.pairs_covered
    return (result & ~pair_labels)

def lenR(predicates):
    R = R_(predicates)
    return R.value_counts()[True]

def lenB(predicates):
    B = B_(predicates)
    return B.value_counts()[True]

# Returns the number of predicates in P that cover each pair_label
def deg(pair_labels, P):
    result = pair_labels.copy()
    result[:] = 0
    for p in P:
        result = result + p.pairs_covered.astype(int)
    return result

def get_t_greedy(P, active_pairs, pair_labels):
    b_by_w_best = float('-inf')
    best_p = None
    for p in P:
        bvar = b(p)
        w = p.weight if p.weight != 0 else 1
        if bvar/w > b_by_w_best:
            best_p = p
            b_by_w_best = bvar/w
    return get_pos_pairs_set(p, pair_labels, active_pairs), p


def make_inactive(active_pairs, set_of_indices_to_deactivate):
    for index, label in enumerate(active_pairs):
        if index in set_of_indices_to_deactivate:
            active_pairs[index] = False
            
def r(p):
    global active_pairs, pair_labels
    counts = (p.pairs_covered & ~pair_labels & active_pairs).value_counts()
    if True in counts:
        return counts[True]
    else:
        print("Predicate {} apparently covers 0 negative pairs".format(p))
        return 0
    
def b(p):
    global active_pairs, pair_labels
    counts = (p.pairs_covered & pair_labels & active_pairs).value_counts()
    if True in counts:
        return counts[True]
    else:
        print("Predicate {} apparently covers 0 positive pairs".format(p))
        return 0

In [178]:
def do_disjunctive_blocking(B, R, P_orig, epsilon, eta, active_pairs, pair_labels):
    
    # Method:
    # 1. Discard from P all predicates p for which r(p) >= eta'
    P = set([p for p in P_orig if r(p) <= eta])
    print('{} predicates discarded, {} retained'.format(len(P_orig)-len(P), len(P)))

    # 2. Check if cover is feasible
    lenBP = lenB(P)
    if len(B) - lenBP > epsilon:
        print("Cover is not feasible, eta too low")
        return P
    print("Cover is feaisble, |B(P)| = {}".format(lenBP))

    # 3. Set gamma (the unit for it is probably number of predicates 
    #                  (though it seems more like root num predicates))
    beta = len(B)
    gamma = math.sqrt(t/math.log(beta))
    print("Gamma set to {}".format(gamma))

    # 4. Discard all r in R covered by more than gamma predicates
    degrees = deg(pair_labels, P)
    print("Calculated the degree of P")
    R_discard = set([r for r in R if degrees[r] > gamma])
    make_inactive(active_pairs, R_discard)
    R = R - R_discard
    print('{} negative pairs discarded'.format(len(R_discard)))

    # 5. Construct an instance of wtd cover set
    # This will be done differently given the current implementation
    for p in P:
        # TODO: Remove after predicate class is re-init
        p.weight = r(p)
        #p.set_weight(pair_labels, active_pairs)
    print('Weights for remaining predicates have been set')

    # 6. Initialize T*, for us it will really be equal to P*
    T_star = set()

    # 7. 
    while len(B) >= epsilon:
        # 8. select the t in T that maximizes b(t)/w(t)
        # TODO: what is get_t_greedy()?
        B_t, p = get_t_greedy(P, active_pairs, pair_labels)
        print("Chose {} by the greedy algorithm.".format(p))

        # 9. Remove the covered pairs from B
        len_prev = len(B)
        B = B - B_t
        len_now = len(B)
        make_inactive(active_pairs, B_t)
        print('New pairs added to covered set'.format(len_prev - len_now))

        # 10. Add the predicate to T*
        T_star.add(p)
        
        # Missing step? Remove p from P
        P.remove(p)

    # 11. Return the P* corresponding to T*
    return T_star

In [156]:
data = read_json_data('data/df_adaptive.json')
data['id'] = data.index
data['tmpkey'] = 1
cols = ['id', 'class', 'tmpkey']
merged = pd.merge(data[cols], data[cols], on='tmpkey')\
           .drop('tmpkey',axis=1)
pair_labels = merged.apply(lambda row : row['class_x'] ==  row['class_y'], axis=1)
print("Computed labels")
print("{} pairs are true out of a total of {}".format(pair_labels.value_counts()[True], len(pair_labels)))

active_pairs = pair_labels.copy()
active_pairs[:] = True

Computed labels
146128 pairs are true out of a total of 3526884


In [37]:
P = define_predicates(data)

Computed labels
146128 pairs are true out of a total of 3526884
processing for predicate (Contains common integer), field (authors)
Elapsed time is 66.20941519737244 seconds.
processing for predicate (Contains common integer), field (title)
Elapsed time is 66.61799788475037 seconds.
processing for predicate (Contains common integer), field (year)
Elapsed time is 66.68908071517944 seconds.
processing for predicate (Contains common integer), field (venue)
Elapsed time is 65.87133002281189 seconds.
processing for predicate (Contains common integer), field (other)
Elapsed time is 66.13545560836792 seconds.
processing for predicate (Contains same or off-by-one integer), field (authors)
Elapsed time is 66.50931978225708 seconds.
processing for predicate (Contains same or off-by-one integer), field (title)
Elapsed time is 66.53623461723328 seconds.
processing for predicate (Contains same or off-by-one integer), field (year)
Elapsed time is 69.06398224830627 seconds.
processing for predicate (

In [157]:
pair_labels_orig = pair_labels.copy()
active_pairs_orig = active_pairs.copy()

In [179]:
pair_labels = pair_labels_orig.copy()
active_pairs = active_pairs_orig.copy()

# coreferent records, represented as indices to the pair_labels Series
B = set([index for index, label in enumerate(pair_labels) if label])

# non-coreferent records
R = set([index for index, label in enumerate(pair_labels) if not label])

epsilon = int(len(B) * .50)    # max uncovered coref pairs
eta  = len(R)    # max pairs any predicate may cover
beta = len(B)    # Number of coreferent pairs
rho  = len(R)    # Number of non-coreferent pairs
t    = len(P)    # Number of predicates under consideration

In [180]:
def get_pos_pairs_set(p, pair_labels, active_pairs):
    return set([index for index, is_true in enumerate(p.pairs_covered & pair_labels & active_pairs) if is_true])

P_star = do_disjunctive_blocking(B, R, P, epsilon, eta, active_pairs, pair_labels)

Predicate Contains same or off-by-one integer apparently covers 0 negative pairs
Predicate Contains same or off-by-one integer apparently covers 0 negative pairs
Predicate Contains same or off-by-one integer apparently covers 0 negative pairs
Predicate Contains same or off-by-one integer apparently covers 0 negative pairs
Predicate Contains same or off-by-one integer apparently covers 0 negative pairs
0 predicates discarded, 35 retained
Cover is feaisble, |B(P)| = 146118
Gamma set to 1.715545424653564
Calculated the degree of P
2629256 negative pairs discarded
Predicate Same 7 first chars apparently covers 0 negative pairs
Predicate Contains common integer apparently covers 0 negative pairs
Predicate Exact match apparently covers 0 negative pairs
Predicate Exact match apparently covers 0 negative pairs
Predicate Exact match apparently covers 0 negative pairs
Predicate Same 7 first chars apparently covers 0 negative pairs
Predicate Contains same or off-by-one integer apparently covers 0

In [144]:
for index, label in enumerate(active_pairs):
    pass